# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import sklearn

from azureml.core import Dataset, Environment, Experiment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl-experiment'

experiment=Experiment(ws, experiment_name)

In [3]:
# create a TabularDataset from a dataset

dataset = Dataset.get_by_name(ws, name='clinical-records')
clinical_records = dataset.to_pandas_dataframe()

# preview the first 3 rows of the dataset
clinical_records.head(3)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [4]:
automl_settings = {
    "task": 'classification',
    "iterations": 30,
    "experiment_timeout_minutes": 15,
    "primary_metric": 'accuracy',
    "training_data": dataset,
    "label_column_name": 'DEATH_EVENT',
    "n_cross_validations": 3,
    "max_concurrent_iterations": 8,
    "max_cores_per_iteration": -1
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(**automl_settings)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=False)

WARNING - Running on local machine. Note that local runs always run synchronously if not using Docker containers, even if you use the parameter 'show_output=False'


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_84f78ff2-94ae-4a43-87e8-acd2cd536a91',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-10-04T07:59:31.465187Z',
 'endTimeUtc': '2020-10-04T08:13:23.650551Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"automl-experiment","subscription_id":"b19c9c81-5f59-4537-b3c0-c1beb163ec22","resource_group":"aml-quickstarts-119684","workspace_name":"quick-starts-ws-119684","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"iterations":30,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":2,"featuriz

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
# Retrieve and save your best automl model.
best_automl_run, best_model = remote_run.get_output()
print(best_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('24',
                                           Pipeline(memory=None,
                                                    steps=[('sparsenormalizer',
                                                            <azureml.automl.runtime.shared.model_wrappers.SparseNormalizer object at 0x7f18545b1cc0>),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.99,
                                                                               importance_type='split',
                                                                               learning_rate=...
             

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [7]:
model = best_automl_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')
model.download() # downlaod the model artifacts with the conda environment YAML file

'outputs'

In [8]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
environment = Environment.from_conda_specification('my-automl-environment', './outputs/conda_env_v_1_0_0.yml')
inference_config = InferenceConfig(entry_script='score.py', environment=environment)

service = model.deploy(
    workspace=ws,
    name="automl-deploy-1",
    models=[model],
    inference_config=inference_config,
    deployment_config=aci_config,
    overwrite=True
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.................................................................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [9]:
service.update(enable_app_insights=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [13]:
ROW_ID = 0
x = clinical_records.iloc[ROW_ID, :-1].to_dict()
y = clinical_records.iloc[ROW_ID, -1]
print(x)

{'age': 75.0, 'anaemia': 0.0, 'creatinine_phosphokinase': 582.0, 'diabetes': 0.0, 'ejection_fraction': 20.0, 'high_blood_pressure': 1.0, 'platelets': 265000.0, 'serum_creatinine': 1.9, 'serum_sodium': 130.0, 'sex': 1.0, 'smoking': 0.0, 'time': 4.0}


In [14]:
import json
import numpy as np
input_payload = json.dumps({
    'data': x
})

output = service.run(input_payload)
print(f"Predicted: {output}\nExpected: {y}")

Predicted: [1]
Expected: 1


TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
logs = service.get_logs()
print(logs)
service.delete()

2020-10-04T08:38:32,774890771+00:00 - rsyslog/run 
2020-10-04T08:38:32,775869877+00:00 - gunicorn/run 
2020-10-04T08:38:32,784964134+00:00 - iot-server/run 
2020-10-04T08:38:32,793008984+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_26ad3f104f91f55fc6f5a4da919b553b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_26ad3f104f91f55fc6f5a4da919b553b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_26ad3f104f91f55fc6f5a4da919b553b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_26ad3f104f91f55fc6f5a4da919b553b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_26ad3f104f91f55fc6f5a4da919b553b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
rsyslogd